# Lab 4 - Klustring och Associationsanalys 


I denna labb ska ni använda två olika typer av oövervakad inlärning för att analysera transaktionsdata.

I del 1 kommer ni analysera data från genom så kallad RFM analysis, vi kommer genomföra analysen med hjälp a en K-means modell i Python för att klustra data beroende på det mönster som uppvisas i data, baserat på just RFM.

I del 2 ska ni få testa på associationsanalys genom att  analysera transaktionsdata från matvaruhandeln, närmare bestämt för en butik kallad *MatFörAlla* för att på detta sätt komma med råd för hur de bör organisera sin butik för att öka försäljningen. 

Till skillnad från flera av de tidigare laborationerna kommer fokus här inte vara så mycket på programmeringen, utan mycket kod kommer redan finnas i laborationen. Fokus är snarare på att förstå de baokmliggande mekanismerna i denna typ av dataanalys som skiljer sig från de vi tidigare tittat på. 


## Del 1 Klustring

Inspirerad av: Khalid i Towards Data Science

I denna del av laborationen ska ni använda ett dataset med data från en onlineaffär för att lista ut vilka olika kunder ni har- Vilka är exempelvis lojala kunder,som ni bör ägna extra energi åt, och vilka är sådana som bara handlar någon enstaka gång och som det inte är värt att lägga tid och pengar på? Vi ska alltså segmentera våra kunder baserat på några olika kriterier. Själva segmenteringen kommer vi utföra med K-means clustering. 

Det finns många olika sätt att segmentera kunder på, några man kan tänka sig är:

* Demografiskt - ålder, kön etc. Demographic — age, gender, socioeconomic status
* Geografiskt - var de borGeographic — where in the world are they?
* **Beteendemässigt - vad gör dina kunder på din sida?** 

I denna laboration  ska vi segmentera kunder baserat på beteende. Ett vanligt sätt att fånga beteende är genom så kallad RFM segmentering, vilket innebär att kunder segmenteras baserat på tre kriterier nämligen *Recency, Frequency and Monetary*, alltså **RFM**. 

* Recency - antalet dagar sedan en kund köpte något.
* Frequency - Hur ofta en kund handlar
* Monetary - För hur mycket har kunden handlat för?





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics

**F1** Ni kan själva hämta det dataset ni ska använda [här](https://archive.ics.uci.edu/ml/datasets/online+retail) där finns också värdefull information om kvaliteten på detta dataset. 

In [ ]:
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
df = df[df['CustomerID'].notna()]

Vi kommer inte använda hela detta dataset då det är extremt stort, vilket gör att det blir för tungt att arbeta med. Istället väljer vi ut 10000 rader som får representera våra kunders beteende.

In [ ]:
df_fix = df.sample(10000, random_state = 42)
df_fix.shape

Som vanligt behöver vi först bekanta oss med den data vi har tillgänglig och även kontrollera om vi behöver åtgärda något innan vi påbörjar analysen. 

Följande punkter är krav som behöver säkerställas för att K-means ska fungera på ett dataset. 

**Inga nullvärden** K-means kan inte hantera nullvärden överhuvudtaget så de måste antingen ersättas eller så måste hela raden tas bort. 

**Endast numeriska värden.** Eftersom K-means använder distans som beräkningssätt för att hitta kluster fungerar inte kategoriska variabler. Dessa behöver därför ersättas, antingen med numeriska värden, om de tillhör ordnialskala (och därmed har ordning) eller med dummyvariabler om de tillhör nominalskala och alltså inte har någon ordning. Mycket viktigt att inte distanser förändras!

**Inga outliers eller brus.** K-means är väldigt känslig för detta.

**Det finns ingen/liten korrelation mellan variablerna** Korrelerade variabler är att betrakta som brus och är inte meningsfulla för algoritmer som ska dela in data i olika segment eftersom de representerar samma karaktäristik hos ett segment. Variabler med hög korrelation mellan varandra bör alltså inte båda inkluderas.

**Inte för många dimensioner.** När antalet dimensioner (variabler) ökar, kommer distansen mellan alla variabler att konvergera till ett konstant värde mellan vilka variabler som helst, därmed kan man inte längre urskilja några kluster. Ju fler dimensioner, desto svårare att hitta kluster alltså. 

Det finns också några till, men de kommer vi till senare.



**F2** 

>
>a. Är det någon av dessa krav som **inte** är uppfyllda? Motivera svaret.
>
>b. Hantera eventuella nullvärden på lämpligt sätt.

### RFM

För att segmentera våra kunder ka vi alltså använda RFM,vilket betyder att vi behöver skapa dessa features (kolumner) för att kunna klustra dem i enlighet med dessa. Vi behöver såklart utgå från det data vi har för att skapa dessa tre kolumner. Detta görs i koden nedan. 

**F3** Kommentera koden så att det är förståeligt vad som händer. 

In [ ]:
from datetime import datetime,timedelta
df_fix["InvoiceDate"] = df_fix["InvoiceDate"].dt.date
df_fix["TotalSum"] = df_fix["Quantity"] * df_fix["UnitPrice"]

snapshot_date = max(df_fix.InvoiceDate) + timedelta(days=1)

customers = df_fix.groupby(['CustomerID']).agg({
   'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

customers.rename(columns = {'InvoiceDate': 'Recency',
                           'InvoiceNo': 'Frequency',
                           'TotalSum': 'MonetaryValue'}, inplace=True)

Förutom de krav som diskuterats ovan kräver också K-means att data skalas och transformeras så att det är normalfördelat och standardiserat. 

**Symetrisk distribution av variabler i data (ingen skevhet).** Transformering av data till en normal distribution medför att outliers och brus från mindre påverkan. 

**Variabler i samma skala** — Alltså ha samma medelvärde och varians, vanligtvis vill man ha värden mellan -1 till 1 (standardiserad data) eller mellan 0 till 1 (normliserad data). För att K-means ska behandla alla attribut som likvärdiga måste de ha samma skala. 

I nedanstående kodblock följer en ananlys för att ta reda på hur våra RFM-variabler ser ut.

Vi startar med att titta på hur de tre variablerna ser ut.

In [ ]:
#Visualisera distributionen av data i våra tre variabler 
fig, ax = plt.subplots(1, 3, figsize=(15,3))
sns.distplot(customers['Recency'], ax=ax[0])
sns.distplot(customers['Frequency'], ax=ax[1])
sns.distplot(customers['MonetaryValue'], ax=ax[2])
plt.tight_layout()
plt.show()

**F4** Hur ser vår data ut? Är de tre variablernas distribution symetrisk?

För att ytterligare undersöka skevheten kan vi också använda pandas inbyggda funktion för det `pd.skew()`. 

Det går inte att säga exakt vilket värde som reflekterar symmentri även fast man tycka att ett värde på 0 torde göra det, men  det kan också implicera att den ena sidan är "tjock" och den andra "tunn". En tumregel är dock att ett positivt värde indikerar en positiv skevhet, (alltså att värdena tenderar att vara fler över 0 jämfört med under 0) medan ett negativt värde indikerar en negativ skevhet.

Anledningar till skevhet kan vara outliers, men det behöver inte vara så. 

In [ ]:
print(customers['Recency'].skew().round(2))
print(customers['Frequency'].skew().round(2))
print(customers['MonetaryValue'].skew().round(2)) 


**F5** Följande kod hanterar de problem som finns gällande skevheten i de tre variablerna. Hur har skevheten hos de tre variablerna förändrats efter transformeringen?

In [ ]:
from scipy import stats
customers_fix = pd.DataFrame()
customers_fix["Recency"] = stats.boxcox(customers['Recency'])[0]
customers_fix["Frequency"] = stats.boxcox(customers['Frequency'])[0]
customers_fix["MonetaryValue"] = pd.Series(np.cbrt(customers['MonetaryValue'])).values
customers_fix.tail()

In [ ]:
print(customers_fix['Recency'].skew().round(2))
print(customers_fix['Frequency'].skew().round(2))
print(customers_fix['MonetaryValue'].skew().round(2)) 

Vi behöver också se till att kravet på variablerna att befinna sig i samma skala uppfylls. Det görs i nedanstående kod. 

**F6** 

>
>a. Vilken typ av transformering har gjorts? Varför?
>
>b. Visualisera resultatet
>
>c. Varför är det så viktigt i K-means att denna pre-processing görs?
>
>d. Vad för struktur är customers_normalized?

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(customers_fix)
customers_normalized = scaler.transform(customers_fix)
print(customers_normalized.mean(axis = 0).round(2))
print(customers_normalized.std(axis = 0).round(2))

Nu är det dags att testa att använda K-means för att segmentera våra kunder

In [ ]:
from sklearn.cluster import KMeans

sse = {} #sum of squared error
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(customers_normalized)
    sse[k] = kmeans.inertia_ # SSE to closest cluster centroid

plt.title('The Elbow Method')
plt.xlabel('k')
plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

**F7** 

>a. Vilket k är lämpligast enligt armbågsmetoden? Varför?
>
>b. Fyll i detta tal i nedanstående kod som skapar modellen genom att ersätta punkterna med rätt siffra.

In [ ]:
model = KMeans(n_clusters= ..., random_state=42)
model.fit(customers_normalized)
model.labels_.shape

### Vad säger vår modell?

För att undersöka vad vi fått för kluster kan vi räkna ut medelvärdena för våra skapade kluster. Först lägger vi dock till en kolumn för våra kluster i vår dataframe.

Vi kan också visualisera våra kluster för att få en tydligare bild av dem.

>
>a. Vad säger dessa värden om våra kunder?
>
>b.Vad betyder värdena i den sista kolumnen, som också är ny?
>
>c. Vad säger visualiseringen?
>
>d.Baserat på vad vi nu vet, döp de kluster som skapats till lämpliga namn och skriv en kort rekommendation till butiken hur de bör hantera dem.

In [ ]:
customers["Cluster"] = model.labels_
customers.head()

In [ ]:
customers.groupby('Cluster').agg({
    'Recency':'mean',
    'Frequency':'mean',
    'MonetaryValue':['mean', 'count']}).round(1)

In [ ]:
df_normalized = pd.DataFrame(customers_normalized, columns=['Recency', 'Frequency', 'MonetaryValue'])
df_normalized['ID'] = customers.index
df_normalized['Cluster'] = model.labels_
df_normalized.head()

In [ ]:
df_nor_melt = pd.melt(df_normalized.reset_index(),
                      id_vars=['ID', 'Cluster'],
                      value_vars=['Recency','Frequency','MonetaryValue'],
                      var_name='Attribute',
                      value_name='Value')
df_nor_melt.head()

In [ ]:
sns.lineplot('Attribute', 'Value', hue='Cluster', data=df_nor_melt)

*Skriv din rekommendation för de namngivna klustren i denna markdown,ersätt denna text.*

## Del 2 Associationsanalys 

Inspirerad av: David Johnsson

Associationsanalys är en oövervakad analysmetod som går ut på att hitta relationer i data. Ett open source bibliotek som har funktioner för att hantera associationsanalys är exempelvis `mlxtend` som vi kommer använda ilaborationen. Läs mer om `mlxtend` [här](https://rasbt.github.io/mlxtend/). 

### Importera bibliotek och läs in data 

In [ ]:
import csv
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib
%matplotlib inline

In [ ]:
with open("groceries.csv") as groceries_file:
    dataset = list(csv.reader(groceries_file))

F1. Ta reda på hur filen ser ut genom att öppna den i en vanlig text editor. Fundera sedan på varför vi inte kommer läsa in den i en dataframe som vi ofta gjort hitills.

>
>a.Vad för typ av data innehåller filen? 
>
>b.Varför är det olämpligt att använda en dataframe för att läsa in filen?
>

Istället för en dataframe kommer vi läsa in filen i en så kallad sparse matrix som vi sedan kan titta på med en dataframe. Nedan är ett exempel på en sparse matrix för 3 olika kunder som köpt 3 produkter var. Varje produkt som finns får sin egen kolumn medan varje rad representerar en shoppingvagn som någon kund köpt. 


| &nbsp; | Produkt 1 | Produkt 2 | Produkt 3 |
|:------:|:---------:|:---------:|:---------:|
| Kund 1 |     0     |     0     |     1     |
| Kund 2 |     1     |     1     |     0     |
| Kund 3 |     0     |     1     |     0     |


>
>c. Vad är fördelen med att använda en sparse matrix jämfört med att läsa in .csv-filen som den är?
>
>d. Vad finns det för nackdel?
>
>e. Hur ser matrisen ut i jämförelse med .csv-filen?


I `mlxtend` finns en `TransactionEncoder()` klass för att skapa en sparse matrix av vår .csv-fil. Nedan kod uför detta.

>
>f. Kommentera koden nedan så att man förstår vad som händer.
>

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
groceries = pd.DataFrame(te_ary, columns=te.columns_)
groceries.head()

### Summera och inspektera transaktionerna. 

Som vanligt behöver vi göra oss familiära med den data vi har, innan vi kan påbörja arbetet med att generera associationsregler.

**F2** Använd lämpliga funktioner som ni lärt er i tidigare laborationer för att skapa en bild av hur er sparse matrix ser ut, besvara också följande frågor. 
>
>a. Beräkna densiteten på matrisen. TIPS! För att beräkna densiteten behöver ni veta hur många värden som innehåller `True` respektive `False`.
>
>b. Hur många produkter finns i ert dataset?
>
>c.Hur många transaktioner finns i ert dataset?
>
>d.Vilka är de 10 vanligaste produkterna i ert dataset? Svaret ska ges i form av en lista med strängar ex. `["potatis", "köttbullar", mjölk,..]`
>
>e.Hur många transaktioner innehåller produkten "soda"?
>
>f.Hur många transaktioner innehåller endast en produkt?
>
>g.Hur många produkter är det i den transaktion som innehåller flest produkter?
>

### Frekvensen av produkter 

**F3** Följande funktion `item_frequency()` beräknar antalet av en viss produkt i förhållande tilldet totala antalet transaktioner i %. 

>
>a. Använd denna  funktion (alltså anropa funktionen, gör inga ändringar i själva funktionen) för att beräkna frekvensen av följande produkter: "whole milk", "butter", "rice". 
>
>b. Använd funktionen för att beräkna frekvensen av de produkter som finns på rad 4,5 och 6. 
>



In [ ]:
def item_frequency(dataset):
    return dataset.sum() / len(dataset) * 100

item_frequency(groceries)

Mer intressant för vår kommande analys är att veta vilka produkter som förekommer fler gånger än ett visst bestämt värde vilken kallas support. 

**F4** Definiera en funktion `item_frequency_plot` som tar ett dataset och ett supportvärde som parametrar och returnerar dessa produkter och deras frekvens i %.

In [ ]:
def item_frequency_plot(dataset, support): 
    
    #er kod här

**F5** Använd din funktion för att plotta ut de produkter som har en support på minst 0.125 i ett stapeldiagram. (TIPS! använd koden nedan och skicka endast med de korrekta parametrarna för att skapa ett stapeldiagram.

In [ ]:
_ = item_frequency_plot().plot.bar()

## Extrahera associationsregler 

*MatFörAlla* börjar bli otåliga och tycker att er analys hitills endast visat på saker de redan vet. De vill att ni ska ta fram ny kunskap somär till nytta för dem om deras kunder.

Det är därmed dax att extrahera associationsregler för att kunna hjälpa *MatFörAlla* att placera sina produkter rätt i butiken. 

Det finns tre olika typer av associeringsregler:*Trivial*, *Unexplainable* och *Actionable* 

* Actionable - Målet med en market basket analysis är qtt hitta associationer som går att agera på, alltså som ger användbar information.  
* Trivial - Regler som är uppenbara och därmed inte intressanta. 
* Unexplainable - Sambandet som regeln står för går inte att förklara utan ytterligare forskning. 

**F6**

>
>a. Förklara varje regeltyp, vad innebär de för vårt arbete? Ge exempel på de tre regeltyperna utifrån domänen matvaruhandel.
>
>b. Ge ett exempel på varje regel utifrån den data ni har fått av *MatFörAlla*
>
>

För att förenkla hur vi kommunicerar kring assosiationsregeler använder vi följande standardiserade sätt att beskriva dem:

$Antecedent \rightarrow Consequent$.

Exempel:

$Toys, wrapping paper \rightarrow Batteries$ 

Ovanstående tolkas som att om du köper leksaker och inslagningspapper ($Antecedent$) är det troligt att du också köper batterier ($Consequent$)


### Att mäta associeringsregler 

För att ta reda på vilka associationsregler som är värdefulla krävs mycket domänkunskap. Det finns dock också några mätvärden som kan användas för att hjälpa till att avgöra kvaliteten på reglerna och för att veta hur mycket vikt vi bör lägga vid en specifik regel. 

Det finns tre huvudsakliga sätt att mäta associeringsregler:

**Support**

Support är antalet transaktioner som innehåller ett specifierat antal produkter. Ju oftare dessa produkter förekommer gemensamt (alltså idetta fallet köpts gemensamt) desto större blir vikten av supporten.

Om transaktionsdata ser ut enligt följande:

```
t1: Beef, Carrot, Milk
t2: Steak, Cheese
t3: Cheese, Flingor
t4: Steak, Carrot, Cheese
t5: Steak, Carrot, Butter, Cheese, Milk
t6: Carrot, Butter, Milk
t7: Carrot, Milk, Butter
```

Skulle supporten för att kombinationen morötter, smör och mjölk köps tillsammans se ut enligt följande:

$$Support(Carrot \land Butter \land Milk) = \frac{3}{7} = 0.43$$

detta på grund av att en kombination av dessa tre produkter förekommer 3 gånger av 7 möjliga transaktioner. 

**Confidence**

Konfidens innebär att om vi har en regel som säger följande:  $Beef, Chicken \rightarrow Apple$ med en konfidens på 33%, så innebär det att om det finns biff och kyckling i någons shoppingvagn så är det 33% chans att det också finns äpplen. 

Konfidensen beräknas exempelvis såhär: 

Givet följande regel: $Butter \rightarrow Milk, Chicken$

$$Butter \rightarrow Milk, Chicken = \frac{Support (Butter \land Milk \land Chicken)}{Support (Butter)}$$

**Lift**

Lift ger oss ett mätvärde på hur bra en regel är, baserat enbart på den högra sidan av en regel(alltså $Consequent$). Detta innebär att exempelvis regler som inkluderar vanliga produkter som $Consequent$ så kommer reglen inte säga någoting av värde. Det är alltså inte meningsfullt att ha mjölk, som är en väldigt vanlig produkt, på den högra sidan i en regel. 

Tumregeln för Lift är följande: 

Om Lift är  $>1$  så är regeln bättre än att gissa.Om Lift är $\leq1$ så är regeln ungefär likvärdig med en ren gissning. 


Exempel:

$$Chicken \rightarrow Milk = \frac{Support (Chicken \land Milk)}{Support(Chicken) \times Support (Milk)} = \frac{(4 / 7)}{(5 / 7) \times (4 / 7)} = 1.4$$

Detta implicerar att  $Chicken \rightarrow Milk$ skulle kunna vara en bra regel eftersom $1.4 > 1$. Om vi ändrar support för hur ofta mjölk inhandlas till $6 / 7$ istället så blir resultatet ett annat.

$$Chicken \rightarrow Milk = \frac{Support (Chicken \land Milk)}{Support(Chicken) \times Support (Milk)} = \frac{(4 / 7)}{(5 / 7) \times (6 / 7)} = 0.933$$

Nu ser samma regel: $Chicken \rightarrow Milk$ inte längre ut som en bra regel eftersom $0.933 < 1$. 

**F7** Förklara de två matematiska symbolerna $\land$ och $\times$, vad betyder de?

### Utföra associationsanalys i Python 

Nu är det dags att hitta associationsregler för vårt dataset. Vi startar med att definiera defaultvärden för support och confidense.

En viktig funktion för att utföra associationsanalys är funktionen `apriori()` som beräknar support (alltsåfrekvens av produkter)på ett liknande sätt som gjordes i början av denna laboration. Funktionen har dock några ytterligare egenskaper som gör den lämplig att använda för våra syften. Exempelvis kan vi ange minsta support i funktionen.

**F8**
>
>a.Testa att använda funktionen `apriori()` med vårt givna dataset, samt ange att minimisupport ska vara 0,5%, spara och skriv ut resultatet i en variabel som heter `frequent_itemsets`. 
>
>b.Förutom att beräkna support,vad gör funktionen `apriori()`?
>
>c.Vad för struktur består resultatet av?
>


In [ ]:
frequent_itemsets = #kod här

Som ni förmodligen sett vid det här laget så skapar funktionen `apriori()` inga regler. För att generera dessa används funktionen `association_rules()` i nedan kod skapas assosiationsregler för hela ert dataset. Det är också möjligt att undersöka de olika mätvärdena (`support`, `confidence` och `lift`) genom att specifiera att dessa ska visas samt vad lägsta tröskelvärdet ska vara. I koden nedan är detta gjort för mätvärdet `confidence`

Om du istället vill skapa regler endast för utvalda delar av ett dataset så är det också möjligt.

**F9**

>a.Utifrån resultatet av funktionen `apriori()`, vad kan vara en lämplig filtrering för att utesluta ointressanta delar av ert dataset innan ni skapar regler? Varför?
>
>b.Filtrera bort irrelevant data och skapa sedan nya associationsregler för ert filtrerade dataset.
>
>c.Undersök samtliga tre mätvärden för de nya associationsreglerna för ditt filtrerade dataset.

In [ ]:
grocery_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
grocery_rules

Det är också möjligt att lägga till kolumner för att räkna ut storleken dvs antalet produkter som är  $Antecendants$ i varje regel:

In [ ]:
grocery_rules["num_antecedents"] = grocery_rules["antecedents"].apply(lambda x: len(x))
grocery_rules

**F10** 

>
>a. hur många regler har skapats totalt?
>
>b.Hur många regler har 3 eller fler $Antecendants$?
>
>c. Vad gör en regel intressant?
>
>d.Finns det någon/några av de regler ni skapat med fler än 3 $Antecendants$ som är värda att undersöka närmare?
>

**F11** 

Det är ofta värdefullt att veta vilka regler som inkluderar specifika produkter. 

>
>a. Varför kan detta vara intressant i vårt sammanhang (alltså *MatFörAlla*)
>
>b.Filtrera ut alla regler som innehåller produkten citrusfrukt som $Antecendant$ .
>
>c.Utforska dessa regler med avseende på konfidence, support och lift.
>
>d.Baserat på era regler, vad för rekommendationer kan ni ge *MatFörAlla* gällande hur de bör placera citrusfrukter i butiken? Förklara svaret.
>
>e.Ta själv fram en artikel som du identifierat som intressant och använd den för att skapa en rekommendation till *MatFörAlla*

**F12**

>a.Givet att *MatFörAlla* också skulle kunna samla in data om vilka kunder som köpt vad istället för bara vad som köpts baserat på ID, vilka
>ytterligare möjlgheter för denna typ av analys kan sådan data ge?
>
>b.En kund köper ett ljus och 20burkar med öl, vilket påverkan har det på er analys och kvaliteten på den? 
>
>c.Hur påverkas er analys av tiden? Vad ställer det för krav på *MatFörAlla*?

När ni är färdiga lämnar ni in i Studium, antingen som html-fil eller med länk till Collaboratory. 

**Lycka till!**
